***
모든 기존 전처리 파일에서 unnamed:0 열 삭제함<br>
train 파일에서 Book-Rating 열 삭제함
***

In [17]:
import re
import pandas as pd
import numpy as np

In [56]:
# 데이터 로드 + 전처리
df_user = pd.read_csv('C:/RecoSys/Data/project/PR_users.csv', encoding='cp949')     #columns=[User-ID, Age_cate, state, country]

df_item = pd.read_csv('C:/RecoSys/Data/project/PR_books.csv', encoding='cp949')     #columns=[ISBN, Book-Title, Book-Author, Year-Of-Publication, Publisher]
df_item['Book-Title'] = df_item['Book-Title'].apply(lambda x : re.sub(r'[^a-z]','', x)) #특수문자 제거
df_item['Book-Author'] = df_item['Book-Author'].apply(lambda x : re.sub(r'[^a-z]','', str(x))) #특수문자 제거
df_item['Publisher'] = df_item['Publisher'].apply(lambda x : re.sub(r'[^a-z]','', x)) #특수문자 제거

df_train1 = pd.read_csv('C:/RecoSys/Data/project/train_1.csv')  #columns=[User-ID, ISBN, Book-Rating]
df_train2 = pd.read_csv('C:/RecoSys/Data/project/train_2.csv')
df_train3 = pd.read_csv('C:/RecoSys/Data/project/train_3.csv')

df_test1 = pd.read_csv('C:/RecoSys/Data/project/test_1.csv')    #columns=[User-ID, ISBN]
df_test1_answer = pd.read_csv('C:/RecoSys/Data/project/test_1_answer.csv') #columns=[Book-Rating]
df_test2 = pd.read_csv('C:/RecoSys/Data/project/test_2.csv')
df_test2_answer = pd.read_csv('C:/RecoSys/Data/project/test_2_answer.csv')
df_test3 = pd.read_csv('C:/RecoSys/Data/project/test_3.csv')
df_test3_answer = pd.read_csv('C:/RecoSys/Data/project/test_3_answer.csv')

# 혹시나 원본 파일로 작업하게 된다면 귀찮지 말라고 남겨둠,,
#df.drop(columns='Unnamed: 0', inplace=True)
#df.drop(columns=['Unnamed: 0', 'Book-Rating'], inplace=True)

In [48]:
def basic(user_id, isbn, rating_matrix, user_mean, item_mean, total_mean):

    con1 = isbn in rating_matrix
    con2 = user_id in rating_matrix

    # 현 item에 대한 본인과 타인의 rating이 있는 경우 -> 타 user의 평점에 현 user의 평가 경향을 고려하여 사용
    if con1 & con2 :

        user_mean = user_mean[user_id]              # 현 user의 rating 평균
        item_mean = item_mean[isbn]                 # 현 item의 rating 평균

        movie_ratings = rating_matrix[isbn] # 현 item의 모든 rating 가져오기   
        others_mean = user_mean.copy()      # 모든 사용자의 rating 평균 가져오기
        
        # 현 item에 대한 rating이 없는 user 삭제
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index   
        movie_ratings = movie_ratings.drop(none_rating_idx)
        others_mean = others_mean.drop(none_rating_idx)
        
        movie_ratings = movie_ratings - others_mean     # 편차 예측치 계산
        prediction = user_mean + movie_ratings.mean()   # 예측값 = 사용자 평균 + 편차의 평균

    # 현 item에 대한 타 user의 rating만 있는 경우 -> 타 user의 평균 평점 사용
    elif con1 :
        prediction = item_mean[isbn]
    
    # 현 user의 타 item에 대한 rating만 있는 경우 -> user의 평균 평점 사용
    elif con2 :
        prediction = user_mean[user_id]
    
    # 현 item과 user에 대한 정보가 모두 없는 경우 -> 평균 평점 사용
    else :
        prediction = total_mean
    
    return prediction

In [20]:
# RMSE 계산을 위한 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [51]:
def score(df_train, df_test_x, df_test_y) :
    
    x_train = df_train[df_train['Book-Rating'] > 0]  # explicit rating만 선정
    total_mean = x_train['Book-Rating'].mean()  # 모든 user와 item의 평균

    rating_matrix = x_train.pivot(values='Book-Rating', index='User-ID', columns='ISBN')
    user_mean = rating_matrix.mean(axis=1)    # user 각각의 rating 평균 계산
    item_mean = rating_matrix.mean(axis=0)    # item 각각의 rating 평균 계산

    id_pairs = zip(df_test_x['User-ID'], df_test_x['ISBN'])
    y_pred = np.array([basic(user, isbn, rating_matrix, user_mean, item_mean, total_mean) for (user, isbn) in id_pairs])
    y_true = np.array(df_test_y['Book-Rating'])

    return RMSE(y_true, y_pred)


In [60]:
score1 = score(df_train1, df_test1, df_test1_answer)
score2 = score(df_train2, df_test2, df_test2_answer)
score3 = score(df_train3, df_test3, df_test3_answer)
score4 = (score1 + score2 + score3) / 3

print(round(score1,4), round(score2, 4), round(score3, 4), '=>', round(score4, 4))

1.9032 1.9784 1.9691 => 1.9502
